# Import the Libraries

In [2]:
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim
import gensim
from gensim.models import Word2Vec
import nltk
from nltk import sent_tokenize, word_tokenize

# from nltk.stem.snowball import SnowballStemmer
# from nltk.stem.wordnet import WordNetLemmatizer
# import spacy

from nltk.corpus import stopwords
import re
import gensim.corpora as corpora
from pprint import pprint


/Users/kellylam/opt/anaconda3/lib/python3.8/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


# Import Data

In [4]:
df = pd.read_csv('Glassdoor_clean_data.csv')
df.head()

/Users/kellylam/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Job Title,Company,Rating,Location,City,State,Salary Estimate,Min Salary (Thousands),Max Salary (Thousands),Average Salary (Thousands),Size,Industry,Job Description
0,Data Analyst,Bell Partners,0.0,"Raleigh, NC",Raleigh,NC,$40K - $73K (Glassdoor est.),40,73,56.5,1001 to 5000 Employees,Real Estate,Data Analyst\nCorporate Headquarters\n12575 Ul...
1,Data Analyst,Uline,3.6,"Pleasant Prairie, WI",Pleasant Prairie,WI,$65K - $79K (Employer est.),65,79,72.0,5001 to 10000 Employees,NaN,Data Analyst\nCorporate Headquarters\n12575 Ul...
2,Data Analyst (South of Milwaukee),Uline,3.6,"Milwaukee, WI",Milwaukee,WI,$65K - $79K (Employer est.),65,79,72.0,5001 to 10000 Employees,NaN,Data Analyst\nCorporate Headquarters\n12575 Ul...
3,Operations Data Analyst,Plante Moran,0.0,"Kalamazoo, MI",Kalamazoo,MI,$36K - $62K (Glassdoor est.),36,62,49.0,1001 to 5000 Employees,Accounting & Legal,Make your mark. With more than $17.2 billion o...
4,Data Analyst,Sperry Rail Inc.,2.7,"Olathe, KS",Olathe,KS,$40K - $69K (Glassdoor est.),40,69,54.5,201 to 500 Employees,Transportation & Logistics,"Data Analyst I: Based in Olathe, Ks. This is a..."


# Data Cleaning

In [5]:
# Getting rid of the paragraph spaces \n.
df['Job Description'] = df['Job Description'].apply(lambda x: x.replace('\n', ' '))
df['Job Description']

/Users/kellylam/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0      Data Analyst Corporate Headquarters 12575 Ulin...
1      Data Analyst Corporate Headquarters 12575 Ulin...
2      Data Analyst Corporate Headquarters 12575 Ulin...
3      Make your mark. With more than $17.2 billion o...
4      Data Analyst I: Based in Olathe, Ks. This is a...
                             ...                        
886    Equivalent Experience  Description: Our large ...
887    Growing primarily in the finance vertical, Har...
888    Secure our Nation, Ignite your Future  ManTech...
889    Team Overview This position is within Home Len...
890    Location: Richmond, VA/ 12+ Months Contract  W...
Name: Job Description, Length: 891, dtype: object

# Preprocessing

In [19]:
all_sentences = []
for i in range(len(df['Job Description'])):
    text = re.sub(r'[^a-zA-Z\s]+',' ', df['Job Description'][i]) # Remove numbers and punctuations
    text = re.sub(r'\s+',' ',text) # Remove multiple space to single space
    text = text.lower() # Making all words lower case
    sentences = nltk.sent_tokenize(text) # Preparing the data
    all_sentences.extend(sentences)

all_sentences[:2]


/Users/kellylam/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['data analyst corporate headquarters uline dr pleasant prairie wi we re passionate about our products because our customers rely on them to run their businesses better together than apart this position is on site and we are looking for good people who share our passion uline is proud to operate as a drug free workplace all new hires must complete a pre employment drug screening position responsibilities write sql queries to retrieve sales customer and product data perform ad hoc analysis and create reports to illustrate performance answer business questions and provide recommendations apply basic and intermediate statistical methods e g data aggregation regression analysis time series analysis etc to solve business questions collaborate with pricing team members to create reports analyses and presentations build relationships with the merchandising and logistics teams to develop trust and understand processes create what if scenarios to quantify the impact due to catalog price or vend

In [20]:
# Tokenizing 
all_sentences = [nltk.word_tokenize(all_sentence) for all_sentence in all_sentences]

/Users/kellylam/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
# Removing stop words (the most common english language daily words)
for i in range(len(all_sentences)):
    all_sentences[i] = [word for word in all_sentences[i] if word not in stopwords.words('english')]

/Users/kellylam/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Training Word2Vec

In [22]:
model_1 = Word2Vec(all_sentences, min_count=1)
words = model_1.wv.vocab
# words

/Users/kellylam/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Retriving Most Similar Word/Similar By Word

In [88]:
similar = model_1.wv.similar_by_word('sql', 50)
similar

/Users/kellylam/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[('tableau', 0.9912209510803223),
 ('r', 0.9880956411361694),
 ('python', 0.9853832125663757),
 ('programming', 0.9846708178520203),
 ('server', 0.9843710660934448),
 ('sas', 0.982599139213562),
 ('languages', 0.9812554121017456),
 ('proficient', 0.9776793122291565),
 ('etc', 0.9768024682998657),
 ('proficiency', 0.9762797951698303),
 ('matlab', 0.9738620519638062),
 ('query', 0.9717943668365479),
 ('vertica', 0.9704968929290771),
 ('familiarity', 0.9695672988891602),
 ('language', 0.968237042427063),
 ('datastudio', 0.9678745865821838),
 ('looker', 0.9662032127380371),
 ('dynamics', 0.9638185501098633),
 ('tsql', 0.9628200531005859),
 ('spreadsheet', 0.9623554348945618),
 ('altryx', 0.9612855911254883),
 ('relational', 0.9610143899917603),
 ('spss', 0.9606311321258545),
 ('coding', 0.9602651596069336),
 ('intermediate', 0.9601835012435913),
 ('schemas', 0.9598783254623413),
 ('postgres', 0.959714949131012),
 ('scripting', 0.9589242339134216),
 ('knowledge', 0.9572758078575134),
 ('bi'

# LDA/ Topic Modeling

In [23]:
# Create Dictionary
id2word = corpora.Dictionary(all_sentences)

# Create Corpus
texts = all_sentences

# Term Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# corpus

/Users/kellylam/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [83]:
num_topics = 20
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus = corpus, id2word = id2word, num_topics = num_topics)
# Print
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

/Users/kellylam/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.031*"data" + 0.011*"experience" + 0.010*"business" + 0.008*"team" + '
  '0.007*"work" + 0.006*"skills" + 0.005*"analytics" + 0.005*"analyst" + '
  '0.005*"support" + 0.004*"sql"'),
 (1,
  '0.023*"data" + 0.012*"experience" + 0.008*"analysis" + 0.007*"work" + '
  '0.007*"business" + 0.005*"analyst" + 0.004*"skills" + 0.004*"information" + '
  '0.004*"status" + 0.004*"company"'),
 (2,
  '0.039*"data" + 0.018*"experience" + 0.009*"business" + 0.008*"work" + '
  '0.007*"skills" + 0.007*"ability" + 0.006*"team" + 0.006*"management" + '
  '0.006*"years" + 0.005*"analysis"'),
 (3,
  '0.038*"data" + 0.011*"experience" + 0.009*"business" + 0.007*"analysis" + '
  '0.006*"work" + 0.006*"team" + 0.005*"analyst" + 0.005*"information" + '
  '0.005*"ability" + 0.004*"analytics"'),
 (4,
  '0.047*"data" + 0.009*"experience" + 0.008*"business" + 0.008*"team" + '
  '0.007*"management" + 0.006*"work" + 0.006*"analysis" + 0.005*"analyst" + '
  '0.005*"reports" + 0.005*"support"'),
 (5,
  '0.018*"

# LDA Visualization

In [84]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/kellylam/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.013908 -0.000355       1        1  24.658297
2     -0.015597 -0.000905       2        1  16.957580
15    -0.003445 -0.000315       3        1   7.537779
0     -0.004498  0.012440       4        1   7.513580
8     -0.006777 -0.002934       5        1   5.789508
18    -0.006837  0.000947       6        1   5.497379
4     -0.018802 -0.003759       7        1   5.022375
17    -0.011488  0.001197       8        1   3.833776
6     -0.005853  0.002326       9        1   3.144883
16     0.014618 -0.004519      10        1   3.017868
11    -0.002906  0.001948      11        1   2.713853
1      0.013120 -0.018792      12        1   2.518401
3     -0.001924 -0.003934      13        1   1.930538
12    -0.001049  0.019550      14        1   1.923032
5      0.005011 -0.007670      15        1   1.915317
10     0.006714 -0.004306      16        1   1.864883
13     0.011267  0.001863      17        1   1.687580
19     0.006356 -0.001636      18        1   1.536621
7      0.006942 -0.004962      19        1   0.658402
14     0.029055  0.013818      20        1   0.278348, topic_info=           Term         Freq        Total Category  logprob  loglift
50         data  8728.000000  8728.000000  Default  30.0000  30.0000
289  experience  2944.000000  2944.000000  Default  29.0000  29.0000
24     business  2491.000000  2491.000000  Default  28.0000  28.0000
419        work  2029.000000  2029.000000  Default  27.0000  27.0000
222        team  1627.000000  1627.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
181     quality     1.816069   783.091190  Topic20  -5.9607  -0.1825
7      analysis     2.101831  1465.562133  Topic20  -5.8145  -0.6631
164   preferred     1.666511   640.697511  Topic20  -6.0466  -0.0678
190     reports     1.740787   849.650581  Topic20  -6.0030  -0.3064
922   knowledge     1.714601   782.508118  Topic20  -6.0182  -0.2393

[1541 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
10190      4  0.373446       ab
10190      7  0.373446       ab
648        1  0.225436  ability
648        2  0.231764  ability
648        3  0.079100  ability
...      ...       ...      ...
10014      1  0.107272       zu
10014      2  0.107272       zu
10014      3  0.107272       zu
10014      4  0.321816       zu
10014      9  0.107272       zu

[4858 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 3, 16, 1, 9, 19, 5, 18, 7, 17, 12, 2, 4, 13, 6, 11, 14, 20, 8, 15])

# Possibly revisit to 
* Lemmatize 
* Build LDA Mallet Model
    * Find optimal number of topics
    * Topic keywords